In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
# Create the input datasets
raw = torch.randn(500, 3)

In [3]:
# Adjust x0 variance and mean
raw[:,0] /= 1
raw[:,0] += 0 #4

# Adjust x1 variance and mean
raw[:,1] /= 1 #3
raw[:,1] += 0 #-1

# Up noise of x2 and convert into y
raw[:,2] *= 0 #2

raw[:,2] = torch.sigmoid( 0.2 * raw[:,0] + 1 * raw[:,1] + raw[:,2] )
raw[:,2] = (raw[:,2] > 0.7)

In [4]:
raw[:,2].mean()

0.18

In [146]:
class UltraSimpleNet(nn.Module):
    def __init__(self):
        super(UltraSimpleNet, self).__init__()
        self.fc = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [147]:
unet = UltraSimpleNet()
optimizer = optim.SGD(unet.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [161]:
epoch = range(10)
pred_sum = 0.0
act_sum = 0.0
count = 0.0

for e in epoch:
    for i in range(len(raw)):
        row = raw[i:i+1,:]
        
        out = unet(Variable(row[:,0:2]))
        pred = torch.exp(out)
        act = Variable(row[0,2:3])
        
        pred_sum += pred.data[0][1]
        act_sum += act.data[0]
        count += 1.0
        
        optimizer.zero_grad()
        loss = criterion(out, act.long())
        loss.backward()
        optimizer.step()

In [148]:
indx = 7

pred = torch.exp(unet(Variable(raw[indx:indx+1,0:2])))
act = raw[indx:indx+1,2:3]

print("Pred: " + str(pred))
print("Act: " + str(act))

Pred: Variable containing:
 0.6955  0.3045
[torch.FloatTensor of size 1x2]

Act: 
 1
[torch.FloatTensor of size 1x1]



In [162]:
print("Pred: " + str(pred_sum / count))
print("Act: " + str(act_sum / count))

Pred: 0.18432340226806135
Act: 0.18


In [151]:
pred.data[0] #[1]


 0.6955
 0.3045
[torch.FloatTensor of size 2]

In [134]:
i = 0
row = raw[i:i+1,:]
        
out = unet(Variable(row[:,0:2]))
pred = torch.exp(out)
act = Variable(row[0,2:3])

print(out)

Variable containing:
 0  0
[torch.FloatTensor of size 1x2]



In [ ]:
pred_sum += pred.data[0][1]
act_sum += act.data[0]
count += 1.0

In [112]:
optimizer.zero_grad()
loss = criterion(out, act.long())
loss.backward()
#pred.backward(pred if act == 1 else 1-pred)        
optimizer.step()


 1
[torch.FloatTensor of size 1]

In [145]:
F.log_softmax(unet.fc(Variable(row[:,0:2])), dim=1)

Variable containing:
-0.3419 -1.2393
[torch.FloatTensor of size 1x2]

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [92]:
x0 = Variable(torch.randn(2,1), requires_grad = True)
x1 = Variable(torch.randn(2,1), requires_grad = True)

print("x0: ")
print(x0)
print("x1: ")
print(x1)

x0: 
Variable containing:
-0.6364
 1.8408
[torch.FloatTensor of size 2x1]

x1: 
Variable containing:
 0.5837
 0.7160
[torch.FloatTensor of size 2x1]



In [93]:
y = x0 + 0.2*x1
print("y: ")
print(y)

y: 
Variable containing:
-0.5196
 1.9840
[torch.FloatTensor of size 2x1]



In [94]:
z = y * y / 3
print("z: ")
print(z)

z: 
Variable containing:
 0.0900
 1.3121
[torch.FloatTensor of size 2x1]



In [95]:
out = z.mean()
print("out: ")
print(out)

out: 
Variable containing:
 0.7011
[torch.FloatTensor of size 1]



In [104]:
out.backward(torch.FloatTensor([0]))